# Neural Network - Digital Recognition

The subject today is digit recognition using neural networks. More precisely, the task is to train a neural network so that it finds the digit in a given image of handwritten digit.
The first step is to split the MNIST dataset into a training set, a validation set, and a test set. During the training phase, the neural network is trained on the training set. Still during the training phase, the recognition score of the neural network is checked on a subset of the training set and on the validation set in order to detect potential overfitting. During the test set, the true classification score of the fully-connected neural network is computed on the test set.

In [1]:
# Needed packages
import os
import numpy as np
import random as rd
import tensorflow as tf
import tools.tools as tls
from tqdm import tqdm
import matplotlib.pyplot as plt

## I - Process the data and visualisation

### Import data

In [2]:
path = os.getcwd() + '/mnist/data/'

train_img = np.load(path+'training_images.npy')
train_labels = np.load(path+'training_labels.npy')

valid_img = np.load(path+'validation_images.npy')
valid_labels = np.load(path+'validation_labels.npy')

test_img = np.load(path+'test_images.npy')
test_labels = np.load(path+'test_labels.npy')

### Process the data

In [3]:
# img
train_img_new = train_img.reshape(train_img.shape[0],
                                  train_img.shape[1]**2).astype('uint64')
valid_img_new = valid_img.reshape(valid_img.shape[0],
                                  valid_img.shape[1]**2).astype('uint64')
test_img_new = test_img.reshape(test_img.shape[0],
                                test_img.shape[1]**2).astype('uint64')

# label
train_lab_new = np.array([np.argmax(train_labels[y])
                          for y in range(len(train_labels))])
valid_lab_new = np.array([np.argmax(valid_labels[y])
                          for y in range(len(valid_labels))])
test_lab_new = np.array([np.argmax(valid_labels[y])
                         for y in range(len(valid_labels))])

### Random permutation of the train set (for the training phase of the neural network)

In [4]:
permut_index = np.random.permutation(train_img_new.shape[0])

train_img = train_img[permut_index, :, :]
train_img_new = train_img_new[permut_index, :]
train_labels = train_labels[permut_index, :]
train_lab_new = train_lab_new[permut_index]

### Visualisation (exported in the folder 'img_observation')

In [5]:
# Visualisation
def visualize(path, name, X, y, nb_img=36):
    """Allows to vizualise 36 img and the labels"""
    vector_viz = [rd.randint(0, len(X)-1) for y in np.arange(nb_img)]
    # Img
    nb_vertically = int(np.sqrt(nb_img))
    name_img_out = path+'visualize_img' + '_' + str(name) + '.png'
    tls.visualize_grayscale_images(X[vector_viz, :, :, :],
                                   nb_vertically, name_img_out)

    # Labels
    name_lab_out = path+'visualize_labels' + '_' + str(name)
    labels = train_lab_new[vector_viz]
    labels = labels.reshape((nb_vertically, nb_vertically))

    file = open(name_lab_out, "w")
    labels_str = ''
    for line in labels:
        for elt in line:
            labels_str = labels_str + str(elt) + ' '
        labels_str = labels_str + '\n'
    file.write(labels_str)
    file.close()
    print('Visualisation exported')


path = os.getcwd() + '/img_observation/'
X = train_img
y = train_lab_new
visualize(path, 'obs', X, y, nb_img=36)

Visualisation exported


## II - Basic Neural Network

import the model within the file 'NeuralNetwork.py'

### Training the neural network

In [6]:
# Import the model
from NeuralNetwork import neural_net_model

In [7]:
# Input data
nb_images = train_img_new.shape[0]
nb_input = train_img_new.shape[1]
batch_size = 1000
nb_hidden = 16
nb_classes = 10
learning_rate = 0.5
nb_epoch = 10

The following function is used in order to define batches during the training (it is the equivalent of the next batch function of tf)

In [8]:
def get_batch_idx(i, batch_size, nb_images):
    # Input
    max_batch_size = int(nb_images/batch_size)
    j = i % max_batch_size
    return np.arange(j*batch_size, (j+1)*batch_size)

In [9]:
# Initialisation
X = tf.placeholder(tf.float32, [None, nb_input])
Y = tf.placeholder(tf.float32, [None, nb_classes])

p = neural_net_model(X, nb_input, batch_size, nb_hidden, nb_classes,
                     learning_rate)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(p),
                                              reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

correct_pred = tf.equal(tf.argmax(p, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

acc_train = []
loss_train = []

In [10]:
# Training
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())

    try:
        saver.restore(sess, 'mnist_predictions.ckpt')
    except Exception:
        pass

    # train the model mini batch with 100 elements, for 1K times
    for epoch in tqdm(range(nb_epoch)):

        for i in range(batch_size):
            batch_index = get_batch_idx(i, batch_size, nb_images)
            batch_x, batch_y = (train_img_new[batch_index, :],
                                train_labels[batch_index])
            sess.run([train_step], feed_dict={X: batch_x, Y: batch_y})

        # Accuracy and cross entropy - Training phase
        loss, acc = sess.run([cross_entropy, accuracy],
                             feed_dict={X: valid_img_new, Y: valid_labels})
        loss_train.append(loss)
        acc_train.append(acc)
        print("\nEpoch", str(epoch), "Loss :", str(loss),
              "Accuracy :", str(acc))

    # Test
    # evaluate the accuracy of the model
    test_accuracy = sess.run(accuracy, feed_dict={X: test_img_new,
                                                  Y: test_labels})
    print("\nAccuracy on test set:", acc)

    # pred
    pred = sess.run(p, feed_dict={X: test_img_new, Y: test_labels})
    pred = np.array([np.argmax(pred[y]) for y in range(len(pred))])
    labels = np.array([np.argmax(test_labels[y])
                       for y in range(len(test_labels))])

    print("\nErrors (%):", np.sum(labels != pred)/len(pred), '%')

    # Save the model
    '''if input('Save model ? [Y/N]') == 'Y':
        import os
        saver.save(sess, os.getcwd() +
                   '/nn_saved_sessions/mnist_predictions.ckpt')
        print('Model Saved')'''

    # Close the session
    sess.close()

 10%|█         | 1/10 [00:04<00:38,  4.24s/it]


Epoch 0 Loss : 0.7467897 Accuracy : 0.777


 20%|██        | 2/10 [00:08<00:33,  4.22s/it]


Epoch 1 Loss : 0.6626388 Accuracy : 0.779


 30%|███       | 3/10 [00:12<00:29,  4.22s/it]

...


Epoch 7 Loss : 0.42350593 Accuracy : 0.882


 90%|█████████ | 9/10 [00:37<00:04,  4.21s/it]


Epoch 8 Loss : 0.3957326 Accuracy : 0.891


100%|██████████| 10/10 [00:42<00:00,  4.20s/it]


Epoch 9 Loss : 0.38083085 Accuracy : 0.893

Accuracy on test set: 0.893

Errors (%): 0.105 %


### Visualisation

#### Accuracy

In [13]:
# Visualisation of the accuracy
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), acc_train, label='accuracy', color='blue')
plt.title('Training phase')
plt.ylabel('accuracy - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

#### Cross entrepoy

In [ ]:
# Visualisation of the loss
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), loss_train, label='loss', color='red')
plt.title('Training phase')
plt.ylabel('loss - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

### Observations

## III - Evolved Neural Network

In [14]:
from NeuralNetwork import neural_net_model_evol

In [15]:
# Input data
nb_images = train_img_new.shape[0]
nb_input = train_img_new.shape[1]
batch_size = 1000
nb_hidden1 = 64
nb_hidden2 = 64
nb_classes = 10
nb_epoch = 20

# Drop out level
prob_1 = 0.7
prob_2 = 0.7

The following function is used in order to define batches during the training (it is the equivalent of the next batch function of tf)

In [16]:
def get_batch_idx(i, batch_size, nb_images):
    # Input
    max_batch_size = int(nb_images/batch_size)
    j = i % max_batch_size
    return np.arange(j*batch_size, (j+1)*batch_size)

In [17]:
# Initialisation
X = tf.placeholder(tf.float32, [None, nb_input])
Y = tf.placeholder(tf.float32, [None, nb_classes])
keep_prob_1 = tf.placeholder(tf.float32)
keep_prob_2 = tf.placeholder(tf.float32)

p = neural_net_model_evol(X, nb_input, nb_hidden1, nb_hidden2, nb_classes,
                          keep_prob_1, keep_prob_2)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(p),
                                              reduction_indices=[1]))

correct_pred = tf.equal(tf.argmax(p, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

acc_train = []
loss_train = []
learn_rate = []

We then define a piecewise constant for learning rate in order not to have a constant value through the training phase

In [18]:
# Piecewise constant
min_lr = 0.1
max_lr = 0.5
nb_values = 20

global_step = tf.Variable(0, trainable=False)
boundaries = list(np.linspace(batch_size,
                              batch_size*nb_epoch, nb_values,
                              dtype=np.int32)[:-1])
values = list(np.round(np.linspace(max_lr, min_lr, nb_values), 2))
learning_rate_pc = tf.train.piecewise_constant(global_step, boundaries, values)

j = 0
# Passing global_step to minimize() will increment it at each step.
learning_step_pc = tf.train.GradientDescentOptimizer(learning_rate_pc).minimize(cross_entropy,
                                                                 global_step=global_step)

In [20]:
# Training
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())

    try:
        saver.restore(sess, 'mnist_predictions.ckpt')
    except Exception:
        pass

    # train the model mini batch with 100 elements, for 1K times
    for epoch in tqdm(range(nb_epoch)):

        for i in range(batch_size):
            # Update learning rate
            l_rate = sess.run(learning_rate_pc, {global_step: j})
            j += 1

            batch_index = get_batch_idx(i, batch_size, nb_images)
            batch_x, batch_y = (train_img_new[batch_index, :],
                                train_labels[batch_index])
            sess.run([learning_step_pc], feed_dict={X: batch_x, Y: batch_y,
                                                    keep_prob_1: prob_1,
                                                    keep_prob_2: prob_2})

        # Accuracy and cross entropy - Training phase
        loss, acc = sess.run([cross_entropy, accuracy],
                             feed_dict={X: valid_img_new, Y: valid_labels,
                                        keep_prob_1: 1.0,
                                        keep_prob_2: 1.0})
        loss_train.append(loss)
        acc_train.append(acc)
        learn_rate.append(l_rate)
        print("\nEpoch", str(epoch), "Loss :", str(loss),
              "Accuracy :", str(acc), "Learning rate:", l_rate)

    # Test
    # evaluate the accuracy of the model
    test_accuracy = sess.run(accuracy, feed_dict={X: test_img_new,
                                                  Y: test_labels,
                                                  keep_prob_1: 1.0,
                                                  keep_prob_2: 1.0})
    print("\nAccuracy on test set:", acc)

    # pred
    pred = sess.run(p, feed_dict={X: test_img_new, Y: test_labels,
                                  keep_prob_1: 1.0, keep_prob_2: 1.0})
    pred = np.array([np.argmax(pred[y]) for y in range(len(pred))])
    labels = np.array([np.argmax(test_labels[y])
                       for y in range(len(test_labels))])

    print("\nErrors (%):", np.sum(labels != pred)/len(pred)*100, '%')

    # Save the model
    '''if input('Save model ? [Y/N]') == 'Y':
        import os
        saver.save(sess, os.getcwd() +
                   '/nn_saved_sessions/mnist_predictions.ckpt')
        print('Model Saved')'''

    # Close the session
    sess.close()

  5%|▌         | 1/20 [00:15<04:59, 15.77s/it]


Epoch 0 Loss : 0.8074657 Accuracy : 0.78 Learning rate: 0.48


 10%|█         | 2/20 [00:29<04:33, 15.19s/it]


Epoch 1 Loss : 0.61264664 Accuracy : 0.826 Learning rate: 0.46


 15%|█▌        | 3/20 [00:43<04:11, 14.79s/it]

...


Epoch 17 Loss : 0.2945744 Accuracy : 0.911 Learning rate: 0.12


 95%|█████████▌| 19/20 [04:28<00:14, 14.01s/it]


Epoch 18 Loss : 0.28958887 Accuracy : 0.909 Learning rate: 0.1


100%|██████████| 20/20 [04:42<00:00, 14.01s/it]


Epoch 19 Loss : 0.2905008 Accuracy : 0.912 Learning rate: 0.1

Accuracy on test set: 0.912

Errors (%): 8.15 %


### Visualisation

#### Accuracy

In [21]:
# Visualisation of the accuracy
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), acc_train, label='accuracy', color='blue')
plt.title('Training phase')
plt.ylabel('accuracy - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

#### Loss

In [22]:
# Visualisation of the loss
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), loss_train, label='loss', color='red')
plt.title('Training phase')
plt.ylabel('loss - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

#### Learning rate

In [23]:
# Visualisation of the learning rate
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), learn_rate, label='learning_rate', color='green')
plt.title('Training phase')
plt.ylabel('learning rate - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

### Observations

### We will now use a exponential decay for the learning rate

In [27]:
acc_train = []
loss_train = []
learn_rate = []

In [24]:
# Exponential decay of the learning rate
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.7
learning_rate_ed = tf.train.exponential_decay(starter_learning_rate,
                                              global_step, batch_size, 0.69,
                                              staircase=True)
j = 0
# Passing global_step to minimize() will increment it at each step.
learning_step_ed = tf.train.GradientDescentOptimizer(learning_rate_ed).minimize(cross_entropy,
                                                   global_step=global_step)

In [25]:
# initialize variables and session
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())

    try:
        saver.restore(sess, 'mnist_predictions.ckpt')
    except Exception:
        pass

    # train the model mini batch with 100 elements, for 1K times
    for epoch in tqdm(range(nb_epoch)):

        for i in range(batch_size):
            # Update learning rate
            l_rate = sess.run(learning_rate_ed, {global_step: j})
            j += 1

            batch_index = get_batch_idx(i, batch_size, nb_images)
            batch_x, batch_y = (train_img_new[batch_index, :],
                                train_labels[batch_index])
            sess.run([learning_step_ed], feed_dict={X: batch_x, Y: batch_y,
                                                    keep_prob_1: prob_1,
                                                    keep_prob_2: prob_2})

        # Accuracy and cross entropy - Training phase
        loss, acc = sess.run([cross_entropy, accuracy],
                             feed_dict={X: valid_img_new, Y: valid_labels,
                                        keep_prob_1: 1.0,
                                        keep_prob_2: 1.0})
        loss_train.append(loss)
        acc_train.append(acc)
        learn_rate.append(l_rate)
        print("\nEpoch", str(epoch), "Loss :", str(loss),
              "Accuracy :", str(acc), "Learning rate:", l_rate)

    # Test
    # evaluate the accuracy of the model
    test_accuracy = sess.run(accuracy, feed_dict={X: test_img_new,
                                                  Y: test_labels,
                                                  keep_prob_1: 1.0,
                                                  keep_prob_2: 1.0})
    print("\nAccuracy on test set:", acc)

    # pred
    pred = sess.run(p, feed_dict={X: test_img_new, Y: test_labels,
                                  keep_prob_1: 1.0, keep_prob_2: 1.0})
    pred = np.array([np.argmax(pred[y]) for y in range(len(pred))])
    labels = np.array([np.argmax(test_labels[y])
                       for y in range(len(test_labels))])

    print("\nErrors (%):", np.sum(labels != pred)/len(pred)*100, '%')

    # Save the model
    '''if input('Save model ? [Y/N]') == 'Y':
        import os
        saver.save(sess, os.getcwd() +
                   '/nn_saved_sessions/mnist_predictions.ckpt')
        print('Model Saved')'''

    # Close the session
    sess.close()

  5%|▌         | 1/20 [00:07<02:18,  7.31s/it]


Epoch 0 Loss : 0.72139025 Accuracy : 0.783 Learning rate: 0.7


 10%|█         | 2/20 [00:14<02:11,  7.29s/it]


Epoch 1 Loss : 0.57925904 Accuracy : 0.818 Learning rate: 0.48299998


 15%|█▌        | 3/20 [00:21<02:03,  7.26s/it]

...


Epoch 17 Loss : 0.40871632 Accuracy : 0.88 Learning rate: 0.0012750656


 95%|█████████▌| 19/20 [02:17<00:07,  7.23s/it]


Epoch 18 Loss : 0.40873408 Accuracy : 0.88 Learning rate: 0.0008797953


100%|██████████| 20/20 [02:24<00:00,  7.23s/it]


Epoch 19 Loss : 0.40874663 Accuracy : 0.879 Learning rate: 0.00060705876

Accuracy on test set: 0.879

Errors (%): 11.87 %


### Visualisation

#### Accuracy

In [26]:
# Visualisation of the accuracy
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), acc_train, label='accuracy', color='blue')
plt.title('Training phase')
plt.ylabel('accuracy - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

ValueError: x and y must have same first dimension, but have shapes (20,) and (40,)

#### Loss

In [ ]:
# Visualisation of the loss
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), loss_train, label='loss', color='red')
plt.title('Training phase')
plt.ylabel('loss - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

#### Learning rate

In [ ]:
# Visualisation of the learning rate
plt.figure(figsize=(10, 6))
plt.plot(np.arange(nb_epoch), learn_rate, label='learning_rate', color='green')
plt.title('Training phase')
plt.ylabel('learning rate - training phase')
plt.xlabel('epochs')
plt.legend(loc='best')
plt.show()

### Observations